In [1]:
import sys

sys.path.insert(0, '../')
import utils.common as cm

import joblib
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

In [2]:
train = cm.get_dataframe()
train.head()

,year,brand,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_city_Lkm,fuel_hwy_Lkm,fuel_comb_Lkm,fuel_comb_mpg,co2
0,2002,FORD,FOCUS SEDAN #,COMPACT,2.0,4,A4,X,9.1,6.8,8.1,35,186
1,2021,Porsche,Panamera Turbo S,Full-size,4.0,8,AM8,Z,15.3,11.2,13.5,21,326
2,2011,INFINITI,EX35 AWD,STATION WAGON - SMALL,3.5,6,AS7,Z,12.4,8.5,10.6,27,244
3,2005,BUICK,RENDEZVOUS AWD,SUV,3.4,6,A4,X,12.7,8.7,10.9,26,251
4,2016,ROLLS-ROYCE,GHOST EWB,FULL-SIZE,6.6,12,AS8,Z,19.2,12.4,16.1,18,373


In [3]:
target = cm.get_target()

In [4]:
features_num = cm.get_features_num(train)
features_num

['year', 'engine_size', 'fuel_city_Lkm', 'fuel_hwy_Lkm', 'co2']

In [5]:
features_log = features_num.copy()
features_log.remove("year")
features_log

['engine_size', 'fuel_city_Lkm', 'fuel_hwy_Lkm', 'co2']

In [6]:
# Selección categoricas 
categorical_cols = cm.get_features_cat(train)
categorical_cols

['brand', 'model', 'vehicle_class', 'transmission', 'fuel_type', 'cylinders']

In [7]:
# Limpieza de strings (en vez de la celda anterior)
cleaner = FunctionTransformer(cm.clean_categoricals)


# Pipeline categoricas
cat_pipeline = Pipeline([
    ("cleaner", cleaner),
    ("impute_mode", SimpleImputer(strategy = "most_frequent")),  # metemos la moda 
    ("ohencoder", OneHotEncoder(handle_unknown = "ignore"))      
])

# -------------------------------------------------------------------------------------------------------

# Transformación logarítmica
log_transformer = FunctionTransformer(func= np.log1p)

# Pipeline para transformación logarítmica y escalado de las numericas
num_pipeline = Pipeline([
    ("log", log_transformer),
    ("scaler", MinMaxScaler()) 
])

# --------------------------------------------------------------------------------------------------------

preprocessing = ColumnTransformer([
        ("cat", cat_pipeline, categorical_cols),
        ("log_and_scaler", num_pipeline, features_log), 
        ("scaler_only", MinMaxScaler(), ["year"]),
        ("exclude", "drop", target)
]) 

# X_train = preprocessing.fit_transform(train)
# print(X_train.shape)

In [8]:
# Pipeline para Linear Regression
lr_pipeline = Pipeline([
    ('Premodelado', preprocessing),   
    ('log_transform_regressor', TransformedTargetRegressor(
    regressor=LinearRegression(),
    func=np.log1p,        
    inverse_func=np.expm1 
))
])

# Pipeline para Random Forest
rf_pipeline = Pipeline([
    ('Premodelado', preprocessing),
    ('log_transform_regressor', TransformedTargetRegressor(
    regressor=RandomForestRegressor(random_state=42),
    func=np.log1p,        
    inverse_func=np.expm1 
))
])

# Pipeline para XGBoost
xgb_pipeline = Pipeline([
    ('Premodelado', preprocessing),
    ('log_transform_regressor', TransformedTargetRegressor(
    regressor=xgb.XGBRegressor(random_state=42),
    func=np.log1p,        
    inverse_func=np.expm1 
))
])

In [9]:
y_train=train[target]

In [10]:
cv = 5

#se aplica la validacion cruzada a cada modelo 
#scoring utilizo la raiz del error cuadratico medio
for name, pipe in zip(["lr_pipeline", "rf_pipeline", "xgb_pipeline"], [lr_pipeline, rf_pipeline, xgb_pipeline]):

    resultado = cross_val_score(pipe, train, y_train, cv = cv, scoring = "neg_root_mean_squared_error")
    print(f"{name}: {-np.mean(resultado):.5f}")
    #print(-resultado)

lr_pipeline: 0.32762
rf_pipeline: 0.19187
xgb_pipeline: 0.24887


In [11]:
# Linear Regression
lr_params = {
    'log_transform_regressor__regressor__fit_intercept': [True, False], # si tiene o no intercept
}

lr_grid = GridSearchCV(
    lr_pipeline, 
    lr_params, 
    cv=cv, 
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Random Forest GridSearchCV
rf_params = {
    'log_transform_regressor__regressor__n_estimators': [100, 200],
    'log_transform_regressor__regressor__max_depth': [None, 10, 50],
    'log_transform_regressor__regressor__min_samples_split': [2, 5], 
}

rf_grid = GridSearchCV(
    rf_pipeline, 
    rf_params, 
    cv=cv, 
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# XGBoost GridSearchCV
xgb_params = {
    'log_transform_regressor__regressor__n_estimators': [50, 200],
    'log_transform_regressor__regressor__learning_rate': [0.05, 0.1],
    'log_transform_regressor__regressor__max_depth': [3, 10],
    'log_transform_regressor__regressor__subsample': [0.8, 1.0]
}

xgb_grid = GridSearchCV(
    xgb_pipeline, 
    xgb_params, 
    cv=cv, 
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

pipe_grids = {"gs_lr": lr_grid,
         "gs_rand_forest": rf_grid,
         "gs_xgb": xgb_grid
         }

In [12]:
%%time

for nombre, grid_search in pipe_grids.items():
    grid_search.fit(train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
CPU times: total: 2min 13s
Wall time: 28min 49s


In [28]:
best_grids = [(i, -j.best_score_) for i, j in pipe_grids.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=True)
best_grids

,Grid,Best score
2,gs_xgb,0.181706
1,gs_rand_forest,0.191871
0,gs_lr,0.327617


In [23]:
best_model = pipe_grids[best_grids.iloc[0,0]]

In [24]:
best_model

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Premodelado',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('cleaner',
                                                                                          FunctionTransformer(func=<function clean_categoricals at 0x000001F22790BA60>)),
                                                                                         ('impute_mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['brand',
                                                                          'model',
                                                                          'vehicle_class',
                                                                          'tran...
                                                                                          num_parallel_tree=None,
                                                                                          random_state=42, ...)))]),
             n_jobs=-1,
             param_grid={'log_transform_regressor__regressor__learning_rate': [0.05,
                                                                               0.1],
                         'log_transform_regressor__regressor__max_depth': [3,
                                                                           10],
                         'log_transform_regressor__regressor__n_estimators': [50,
                                                                              200],
                         'log_transform_regressor__regressor__subsample': [0.8,
                                                                           1.0]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [26]:
xgb_grid.best_params_

{'log_transform_regressor__regressor__learning_rate': 0.05,
 'log_transform_regressor__regressor__max_depth': 10,
 'log_transform_regressor__regressor__n_estimators': 200,
 'log_transform_regressor__regressor__subsample': 1.0}

In [25]:
rf_grid.best_params_

{'log_transform_regressor__regressor__max_depth': None,
 'log_transform_regressor__regressor__min_samples_split': 2,
 'log_transform_regressor__regressor__n_estimators': 100}

In [27]:
# Usando pickle:
# with open('modelo_pipeline.pkl', 'wb') as archivo:
#     pickle.dump(best_model, archivo)

# Usando joblib:
joblib.dump(best_model, cm.MODEL_PATH)

['../models/modelo_pipeline.joblib']